## Tabela Silver para Dimenssão de Envolvidos

#### Tratamento do Dataframe para obter informações desejadas

Aqui vamos filtrar apenas as colunas desejadas e realizar o tratamentod e tipagem e limpeza de dados necessárias

Vamos realizar alguns tratamentos

    Seleção de colunas necessárias
    Remoção de duplicidades aproveitado já o campo "pesid" que vem no arquivo original (Por ser o campo unico de cada envolvido)
    Configurar colunas para o padrão
    Filtrando dados indesejados
    Alteração de tipagem de dados
    Removendo dados nulos
    Salvando arquivo de forma compactada

In [0]:
# Importando Bibliotecas para uso do Spark
from pyspark.sql import SparkSession # Import para inciar o spark
from pyspark.sql.types import * # Todos as configurações do SQL de tipagem no spark
from pyspark.sql.functions import * # Todas as funções de SQL do Spark
from pyspark.sql.window import Window

In [0]:
# Diretorio Bronze, com os dados brutos das causas dos acidentes
Bronze_Acidentes_Causas = '/Volumes/workspace/dw_acidentes/01_bronze/Acidentes_Causas/'

Diretorio_Silver = '/Volumes/workspace/dw_acidentes/02_silver'

In [0]:
# Lê os arquivos Parquet da bronze
df_Acidentes_Causas = spark.read.parquet(Bronze_Acidentes_Causas)

In [0]:
# Selecionado apenas as colunas necessária para criar o modelo Dimenssional de Envolvidos
df_envolvidos = df_Acidentes_Causas.select(['pesid','tipo_envolvido', 'estado_fisico', 'idade', 'sexo'])

In [0]:
# Removendo Duplicadas df de veiculos no campo chave
df_envolvidos = df_envolvidos.dropDuplicates(subset=["pesid"])

In [0]:
# Renomeado a coluna para padrão desejado
df_envolvidos = df_envolvidos.withColumnRenamed('pesid', 'id_envolvido')

In [0]:
# REMOVER VALORES INVÁLIDOS ou nulos ANTES DO CAST(Alteração da Tipagem dos campos)
df_envolvidos = df_envolvidos.filter(
    (col("id_envolvido").isNotNull()) &
    (col("id_envolvido") != "NA")
)

In [0]:
# Configurado o nome das colunas para manter um padrão de minuscula e com "_" no lugar de espaçamentos
# Renomear e substituir espaços por underscore e todas as colunas convertendo para minúsculas
df_envolvidos = df_envolvidos.toDF(
    *[c.lower().replace(" ", "_") for c in df_envolvidos.columns]
)

In [0]:
# Alterando a tipagem para padrões necesários
df_envolvidos = df_envolvidos.withColumn('id_envolvido', col('id_envolvido').try_cast(IntegerType()))
df_envolvidos = df_envolvidos.withColumn('idade', col('idade').try_cast(IntegerType()))

In [0]:
# Retirando os nulos usando como parametro o campo chave pois não pode ser nulo
df_envolvidos = df_envolvidos.dropna(subset='id_envolvido')

In [0]:
# Validando os campos para realizar os tratamentos
df_envolvidos.printSchema()

root
 |-- id_envolvido: integer (nullable = true)
 |-- tipo_envolvido: string (nullable = true)
 |-- estado_fisico: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: string (nullable = true)



In [0]:
# Avaliando os primeiros registros analise rapida.
display(df_envolvidos.limit(5))

id_envolvido,tipo_envolvido,estado_fisico,idade,sexo
889,Passageiro,Ileso,null,Masculino
1561,Passageiro,Óbito,22,Feminino
3297,Passageiro,Lesões Leves,19,Feminino
2371,Passageiro,Ileso,14,Masculino
3372,Condutor,Lesões Leves,30,Masculino


In [0]:
# Salvando o arquivo em parquet com compressão gzip 
df_envolvidos.write.parquet(
    f'{Diretorio_Silver}/Dim_Envolidos', 
    mode="overwrite",         # Aqui defino para sempre sobrescrever o arquivo
    compression="gzip"        # Comando para salvar o arquivo com compactação
)